# Modelo de Predicción de Reportes por Zona

Este notebook implementa un modelo supervisado de Machine Learning para predecir la cantidad de reportes futuros en zonas críticas basado en:

- **Análisis temporal**: Patrones estacionales y semanales
- **Características espaciales**: Ubicación y densidad de zonas
- **Categorización**: Tipos de reportes dominantes
- **Random Forest Regressor**: Para predicciones robustas
- **Sistema de recomendaciones**: Basado en las predicciones

## Características del Modelo

✅ **Predicción supervisada** con Random Forest  
✅ **Ingeniería de características temporales** (seno/coseno)  
✅ **Validación con métricas** (MAE, R²)  
✅ **Predicciones a 4 semanas** futuras  
✅ **Sistema de recomendaciones inteligentes**  
✅ **Clasificación por prioridad** (ALTA/MEDIA/BAJA)

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from datetime import datetime, timedelta
import joblib
import sys
import traceback
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")

In [ ]:
def generar_recomendaciones_inteligentes(predicciones_futuras, analisis_zonas):
    """
    Sistema de recomendaciones basado en predicciones
    
    Parámetros:
    - predicciones_futuras: Dict con predicciones por zona
    - analisis_zonas: DataFrame con información de zonas críticas
    
    Retorna:
    - Lista de recomendaciones ordenadas por prioridad
    """
    recomendaciones = []
    
    for zona_key, predicciones in predicciones_futuras.items():
        zona_id = int(zona_key.split('_')[1])
        zona_info = analisis_zonas.loc[zona_id]
        
        # Extraer reportes predichos
        reportes_predichos = [p['reportes_predichos'] for p in predicciones]
        
        # Calcular tendencia (promedio de las diferencias)
        tendencia = np.mean(np.diff(reportes_predichos)) if len(reportes_predichos) > 1 else 0
        max_reportes = max(reportes_predichos)
        
        # Generar recomendaciones según criterios
        if max_reportes > zona_info['densidad'] * 1.5:
            recomendaciones.append({
                'zona_id': zona_id,
                'mensaje': f"Zona {zona_id}: Se espera un aumento significativo de reportes. Reforzar atención.",
                'prioridad': 'ALTA',
                'max_reportes_esperados': max_reportes,
                'tendencia': tendencia
            })
        elif tendencia > 0:
            recomendaciones.append({
                'zona_id': zona_id,
                'mensaje': f"Zona {zona_id}: Tendencia al alza en reportes. Monitorear de cerca.",
                'prioridad': 'MEDIA',
                'max_reportes_esperados': max_reportes,
                'tendencia': tendencia
            })
        elif max_reportes < zona_info['densidad'] * 0.5:
            recomendaciones.append({
                'zona_id': zona_id,
                'mensaje': f"Zona {zona_id}: Baja incidencia esperada. Recursos mínimos recomendados.",
                'prioridad': 'BAJA',
                'max_reportes_esperados': max_reportes,
                'tendencia': tendencia
            })
    
    # Ordenar por prioridad
    orden_prioridad = {'ALTA': 3, 'MEDIA': 2, 'BAJA': 1}
    recomendaciones.sort(key=lambda x: orden_prioridad.get(x['prioridad'], 0), reverse=True)
    
    return recomendaciones

print("✅ Función de recomendaciones inteligentes definida")

In [ ]:
def crear_modelo_prediccion_reportes(df_resultados, analisis_zonas, grafico_path_base="graficos/", verbose=True):
    """
    Modelo SUPERVISADO para predecir cantidad de reportes por zona
    
    Parámetros:
    - df_resultados: DataFrame con resultados del clustering
    - analisis_zonas: DataFrame con análisis de zonas críticas
    - grafico_path_base: Ruta base para guardar gráficos
    - verbose: Si mostrar mensajes de progreso
    
    Retorna:
    - Diccionario con modelo, predicciones y recomendaciones
    """
    def log_print(mensaje):
        if verbose:
            print(mensaje)
    
    try:
        log_print(f"\n=== MODELO DE PREDICCIÓN DE REPORTES ===")
        
        # Validación de datos de entrada
        if analisis_zonas.empty:
            raise ValueError("El análisis de zonas está vacío")
        
        log_print(f"[ETAPA 1/5] Preparando datos de entrenamiento...")
        
        # Preparar datos de predicción por zona
        datos_prediccion = []
        
        for zona_id in analisis_zonas.index:
            zona = analisis_zonas.loc[zona_id]
            df_zona = df_resultados[df_resultados['cluster'] == zona_id].copy()
            
            if df_zona.empty:
                log_print(f"  ⚠️ Zona {zona_id}: Sin datos de reportes")
                continue
            
            # Procesar fechas
            if 'fecha_creacion' not in df_zona.columns:
                log_print(f"  ⚠️ Zona {zona_id}: Sin columna fecha_creacion")
                continue
                
            df_zona['fecha_creacion'] = pd.to_datetime(df_zona['fecha_creacion'], errors='coerce')
            df_zona = df_zona[df_zona['fecha_creacion'].notna()]
            
            if df_zona.empty:
                log_print(f"  ⚠️ Zona {zona_id}: Sin fechas válidas")
                continue
            
            # Extraer características temporales
            df_zona['semana'] = df_zona['fecha_creacion'].dt.isocalendar().week
            df_zona['mes'] = df_zona['fecha_creacion'].dt.month
            df_zona['año'] = df_zona['fecha_creacion'].dt.year
            
            # Agrupar por período temporal
            agrupado = df_zona.groupby(['año', 'mes', 'semana']).size().reset_index(name='num_reportes')
            
            for _, row in agrupado.iterrows():
                datos_prediccion.append({
                    'zona_id': zona_id,
                    'año': row['año'],
                    'mes': row['mes'],
                    'semana': row['semana'],
                    'num_reportes': row['num_reportes'],
                    'lat_centro': zona['lat_centro'],
                    'lng_centro': zona['lng_centro'],
                    'densidad_zona': zona['densidad'],
                    'categoria_dominante': zona.get('categoria_dominante', 0)
                })
        
        if len(datos_prediccion) < 10:
            raise ValueError("No hay suficientes datos históricos para entrenar el modelo supervisado.")
        
        log_print(f"✓ Datos preparados: {len(datos_prediccion)} registros temporales")
        
        return datos_prediccion  # Continuamos en la siguiente celda
        
    except Exception as e:
        log_print(f"❌ Error en preparación de datos: {str(e)}")
        log_print(f"📋 Traceback: {traceback.format_exc()}")
        return None

print("✅ Función de preparación de datos definida")

In [ ]:
def entrenar_modelo_prediccion(datos_prediccion, verbose=True):
    """
    Entrena el modelo de Random Forest con ingeniería de características
    """
    def log_print(mensaje):
        if verbose:
            print(mensaje)
    
    log_print(f"[ETAPA 2/5] Ingeniería de características...")
    
    df_pred = pd.DataFrame(datos_prediccion)
    
    # Características cíclicas para capturar patrones estacionales
    df_pred['semana_sin'] = np.sin(2 * np.pi * df_pred['semana'] / 52)
    df_pred['semana_cos'] = np.cos(2 * np.pi * df_pred['semana'] / 52)
    df_pred['mes_sin'] = np.sin(2 * np.pi * df_pred['mes'] / 12)
    df_pred['mes_cos'] = np.cos(2 * np.pi * df_pred['mes'] / 12)
    
    # Características adicionales
    df_pred['densidad_normalizada'] = (df_pred['densidad_zona'] - df_pred['densidad_zona'].min()) / (df_pred['densidad_zona'].max() - df_pred['densidad_zona'].min() + 1e-10)
    
    log_print(f"✓ Características generadas: {df_pred.shape[1]} variables")\n    
    # Seleccionar características para el modelo
    features_cols = [
        'semana_sin', 'semana_cos', 'mes_sin', 'mes_cos',
        'lat_centro', 'lng_centro', 'densidad_zona', 'categoria_dominante'
    ]
    
    X = df_pred[features_cols]
    y = df_pred['num_reportes']
    
    if len(X) < 4:
        raise ValueError("No hay suficientes muestras para entrenar el modelo.")
    
    log_print(f"[ETAPA 3/5] Entrenando modelo Random Forest...")
    
    # División temporal (más realista que división aleatoria)
    split_point = int(len(X) * 0.8)
    X_train, X_test = X.iloc[:split_point], X.iloc[split_point:]
    y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]
    
    # Entrenar modelo
    modelo_pred = RandomForestRegressor(
        n_estimators=50,
        max_depth=5,
        min_samples_split=2,
        random_state=42,
        n_jobs=-1  # Usar todos los cores disponibles
    )
    
    modelo_pred.fit(X_train, y_train)
    
    # Evaluación del modelo
    y_pred = modelo_pred.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred) if len(y_test) > 1 else 0.0
    
    log_print(f"🎯 Modelo predictivo entrenado:")
    log_print(f"   MAE: {mae:.2f} reportes")
    log_print(f"   R²: {r2:.3f}")
    log_print(f"   Precisión: {max(0, (1 - mae/y.mean()))*100:.1f}%")
    
    # Importancia de características
    feature_importance = pd.DataFrame({
        'feature': features_cols,
        'importance': modelo_pred.feature_importances_
    }).sort_values('importance', ascending=False)
    
    log_print(f"📊 Top 3 características más importantes:")
    for idx, row in feature_importance.head(3).iterrows():
        log_print(f"   {row['feature']}: {row['importance']:.3f}")
    
    return modelo_pred, features_cols, {'mae': mae, 'r2': r2}, feature_importance

print("✅ Función de entrenamiento definida")

In [ ]:
def generar_predicciones_futuras(modelo_pred, features_cols, analisis_zonas, datos_prediccion, verbose=True):
    """
    Genera predicciones para las próximas 4 semanas
    """
    def log_print(mensaje):
        if verbose:
            print(mensaje)
    
    log_print(f"[ETAPA 4/5] Generando predicciones futuras...")
    
    df_pred = pd.DataFrame(datos_prediccion)
    predicciones_futuras = {}
    
    for zona_id in analisis_zonas.index:
        zona = analisis_zonas.loc[zona_id]
        
        # Obtener últimos datos de la zona
        zona_data = df_pred[df_pred['zona_id'] == zona_id]
        if zona_data.empty:
            continue
            
        ult_año = zona_data['año'].max()
        ult_semana = zona_data['semana'].max()
        ult_mes = zona_data['mes'].max()
        
        predicciones_zona = []
        
        # Predecir próximas 4 semanas
        for i in range(1, 5):
            # Calcular fecha futura
            semana_pred = (ult_semana + i) % 52 or 52
            mes_pred = ult_mes if semana_pred >= ult_semana else (ult_mes % 12) + 1
            año_pred = ult_año if mes_pred >= ult_mes else ult_año + 1
            
            # Preparar características para predicción
            features_pred = {
                'semana_sin': np.sin(2 * np.pi * semana_pred / 52),
                'semana_cos': np.cos(2 * np.pi * semana_pred / 52),
                'mes_sin': np.sin(2 * np.pi * mes_pred / 12),
                'mes_cos': np.cos(2 * np.pi * mes_pred / 12),
                'lat_centro': zona['lat_centro'],
                'lng_centro': zona['lng_centro'],
                'densidad_zona': zona['densidad'],
                'categoria_dominante': zona.get('categoria_dominante', 0)
            }
            
            # Realizar predicción
            X_pred = pd.DataFrame([features_pred])
            reportes_pred = modelo_pred.predict(X_pred)[0]
            
            predicciones_zona.append({
                'año': año_pred,
                'mes': mes_pred,
                'semana': semana_pred,
                'reportes_predichos': max(0, round(reportes_pred)),
                'fecha_estimada': f"Semana {semana_pred}, {año_pred}"
            })
        
        predicciones_futuras[f"zona_{zona_id}"] = predicciones_zona
        
        # Log de predicciones por zona
        total_predicho = sum([p['reportes_predichos'] for p in predicciones_zona])
        log_print(f"  Zona {zona_id}: {total_predicho} reportes predichos en 4 semanas")
    
    log_print(f"✓ Predicciones generadas para {len(predicciones_futuras)} zonas")
    
    return predicciones_futuras

print("✅ Función de predicciones futuras definida")

In [ ]:
def crear_modelo_prediccion_completo(df_resultados, analisis_zonas, grafico_path_base="graficos/", verbose=True):
    """
    Función principal que integra todo el pipeline de predicción
    """
    def log_print(mensaje):
        if verbose:
            print(mensaje)
    
    try:
        log_print(f"\n=== MODELO DE PREDICCIÓN DE REPORTES ===")
        
        # Validación de datos de entrada
        if analisis_zonas.empty:
            raise ValueError("El análisis de zonas está vacío")
        
        log_print(f"[ETAPA 1/5] Preparando datos de entrenamiento...")
        
        # Preparar datos de predicción por zona
        datos_prediccion = []
        
        for zona_id in analisis_zonas.index:
            zona = analisis_zonas.loc[zona_id]
            df_zona = df_resultados[df_resultados['cluster'] == zona_id].copy()
            
            if df_zona.empty:
                continue
            
            # Procesar fechas
            if 'fecha_creacion' not in df_zona.columns:
                continue
                
            df_zona['fecha_creacion'] = pd.to_datetime(df_zona['fecha_creacion'], errors='coerce')
            df_zona = df_zona[df_zona['fecha_creacion'].notna()]
            
            if df_zona.empty:
                continue
            
            # Extraer características temporales
            df_zona['semana'] = df_zona['fecha_creacion'].dt.isocalendar().week
            df_zona['mes'] = df_zona['fecha_creacion'].dt.month
            df_zona['año'] = df_zona['fecha_creacion'].dt.year
            
            # Agrupar por período temporal
            agrupado = df_zona.groupby(['año', 'mes', 'semana']).size().reset_index(name='num_reportes')
            
            for _, row in agrupado.iterrows():
                datos_prediccion.append({
                    'zona_id': zona_id,
                    'año': row['año'],
                    'mes': row['mes'],
                    'semana': row['semana'],
                    'num_reportes': row['num_reportes'],
                    'lat_centro': zona['lat_centro'],
                    'lng_centro': zona['lng_centro'],
                    'densidad_zona': zona['densidad'],
                    'categoria_dominante': zona.get('categoria_dominante', 0)
                })
        
        if len(datos_prediccion) < 10:
            raise ValueError("No hay suficientes datos históricos para entrenar el modelo supervisado.")
        
        log_print(f"✓ Datos preparados: {len(datos_prediccion)} registros temporales")
        
        # Entrenar modelo
        modelo_pred, features_cols, metricas, feature_importance = entrenar_modelo_prediccion(datos_prediccion, verbose)
        
        # Generar predicciones futuras
        predicciones_futuras = generar_predicciones_futuras(modelo_pred, features_cols, analisis_zonas, datos_prediccion, verbose)
        
        # Generar recomendaciones
        recomendaciones = generar_recomendaciones_inteligentes(predicciones_futuras, analisis_zonas)
        
        log_print(f"[ETAPA 5/5] Generando recomendaciones y guardando modelo...")
        
        log_print(f"\n📋 RECOMENDACIONES GENERADAS:")
        for i, rec in enumerate(recomendaciones[:3], 1):
            log_print(f"  {i}. {rec['mensaje']} (Prioridad: {rec['prioridad']})")
        
        # Guardar modelo
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        os.makedirs("modelos", exist_ok=True)
        modelo_pred_path = f"modelos/prediccion_reportes_{timestamp}.joblib"
        
        joblib.dump({
            'modelo_prediccion': modelo_pred,
            'feature_names': features_cols,
            'predicciones_futuras': predicciones_futuras,
            'recomendaciones': recomendaciones,
            'metricas': metricas,
            'feature_importance': feature_importance,
            'metadata': {
                'fecha_entrenamiento': timestamp,
                'tipo': 'prediccion_reportes_v1.0',
                'zonas_analizadas': len(analisis_zonas),
                'registros_entrenamiento': len(datos_prediccion)
            }
        }, modelo_pred_path)
        
        log_print(f"✅ Modelo predictivo guardado: {modelo_pred_path}")
        
        return {
            'success': True,
            'modelo_path': modelo_pred_path,
            'predicciones': predicciones_futuras,
            'recomendaciones': recomendaciones,
            'metricas': metricas,
            'feature_importance': feature_importance
        }
        
    except Exception as e:
        log_print(f"❌ Error en modelo predictivo: {str(e)}")
        log_print(f"📋 Traceback: {traceback.format_exc()}")
        return {
            'success': False,
            'error': str(e),
            'traceback': traceback.format_exc()
        }

print("✅ Función principal completa definida")

In [ ]:
def visualizar_predicciones(predicciones_futuras, analisis_zonas, recomendaciones, save_path="graficos/"):
    """
    Crea visualizaciones de las predicciones y recomendaciones
    """
    os.makedirs(save_path, exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Preparar datos para visualización
    datos_viz = []
    for zona_key, predicciones in predicciones_futuras.items():
        zona_id = int(zona_key.split('_')[1])
        for i, pred in enumerate(predicciones, 1):
            datos_viz.append({
                'zona_id': zona_id,
                'semana_futura': i,
                'reportes_predichos': pred['reportes_predichos'],
                'fecha_estimada': pred['fecha_estimada']
            })
    
    df_viz = pd.DataFrame(datos_viz)
    
    # Crear subplot con múltiples gráficos
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Análisis de Predicciones de Reportes por Zona', fontsize=16, fontweight='bold')
    
    # 1. Predicciones por zona a lo largo del tiempo
    for zona_id in df_viz['zona_id'].unique():
        zona_data = df_viz[df_viz['zona_id'] == zona_id]
        ax1.plot(zona_data['semana_futura'], zona_data['reportes_predichos'], 
                marker='o', label=f'Zona {zona_id}', linewidth=2)
    
    ax1.set_title('Predicciones de Reportes por Zona (4 semanas)', fontsize=12)
    ax1.set_xlabel('Semana Futura')
    ax1.set_ylabel('Reportes Predichos')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Total de reportes predichos por zona
    total_por_zona = df_viz.groupby('zona_id')['reportes_predichos'].sum().sort_values(ascending=False)
    colors = plt.cm.viridis(np.linspace(0, 1, len(total_por_zona)))
    bars = ax2.bar(total_por_zona.index.astype(str), total_por_zona.values, color=colors)
    ax2.set_title('Total de Reportes Predichos por Zona (4 semanas)', fontsize=12)
    ax2.set_xlabel('Zona ID')
    ax2.set_ylabel('Total Reportes Predichos')
    
    # Agregar valores en las barras
    for bar, value in zip(bars, total_por_zona.values):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{int(value)}', ha='center', va='bottom', fontweight='bold')
    
    # 3. Distribución de prioridades de recomendaciones
    prioridades = [rec['prioridad'] for rec in recomendaciones]
    prioridad_counts = Counter(prioridades)
    
    colors_pie = {'ALTA': '#ff4444', 'MEDIA': '#ffaa44', 'BAJA': '#44ff44'}
    colors_list = [colors_pie.get(p, '#cccccc') for p in prioridad_counts.keys()]
    
    wedges, texts, autotexts = ax3.pie(prioridad_counts.values(), 
                                       labels=prioridad_counts.keys(),
                                       autopct='%1.1f%%',
                                       colors=colors_list,
                                       startangle=90)
    ax3.set_title('Distribución de Prioridades de Recomendaciones', fontsize=12)
    
    # 4. Comparación densidad actual vs predicciones
    comparacion_data = []
    for zona_id in analisis_zonas.index:
        if f"zona_{zona_id}" in predicciones_futuras:
            predicciones_zona = predicciones_futuras[f"zona_{zona_id}"]
            total_predicho = sum([p['reportes_predichos'] for p in predicciones_zona])
            densidad_actual = analisis_zonas.loc[zona_id, 'densidad']
            
            comparacion_data.append({
                'zona_id': zona_id,
                'densidad_actual': densidad_actual,
                'total_predicho': total_predicho,
                'ratio': total_predicho / (densidad_actual + 1e-10)
            })
    
    if comparacion_data:
        df_comp = pd.DataFrame(comparacion_data)
        x_pos = np.arange(len(df_comp))
        width = 0.35
        
        bars1 = ax4.bar(x_pos - width/2, df_comp['densidad_actual'], width, 
                       label='Densidad Actual', alpha=0.8, color='skyblue')
        bars2 = ax4.bar(x_pos + width/2, df_comp['total_predicho'], width,
                       label='Total Predicho (4 sem)', alpha=0.8, color='lightcoral')
        
        ax4.set_title('Comparación: Densidad Actual vs Predicciones', fontsize=12)
        ax4.set_xlabel('Zona ID')
        ax4.set_ylabel('Cantidad de Reportes')
        ax4.set_xticks(x_pos)
        ax4.set_xticklabels(df_comp['zona_id'].astype(str))
        ax4.legend()
    
    plt.tight_layout()
    
    # Guardar gráfico
    grafico_path = os.path.join(save_path, f"predicciones_analisis_{timestamp}.png")
    plt.savefig(grafico_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    
    print(f"📊 Visualización guardada en: {grafico_path}")
    
    return grafico_path

print("✅ Función de visualización definida")

In [ ]:
def cargar_modelo_prediccion(modelo_path):
    """
    Carga un modelo de predicción previamente entrenado
    """
    try:
        modelo_data = joblib.load(modelo_path)
        
        print(f"📂 Modelo de predicción cargado desde: {modelo_path}")
        print(f"📅 Fecha de entrenamiento: {modelo_data['metadata']['fecha_entrenamiento']}")
        print(f"🔧 Tipo: {modelo_data['metadata']['tipo']}")
        
        # Mostrar métricas
        print(f"\n📈 Métricas del modelo:")
        for key, value in modelo_data['metricas'].items():
            print(f"  {key}: {value:.4f}")
        
        # Mostrar estadísticas
        if 'metadata' in modelo_data:
            meta = modelo_data['metadata']
            print(f"\n📊 Estadísticas:")
            print(f"  Zonas analizadas: {meta.get('zonas_analizadas', 'N/A')}")
            print(f"  Registros de entrenamiento: {meta.get('registros_entrenamiento', 'N/A')}")
        
        # Mostrar top recomendaciones
        if 'recomendaciones' in modelo_data and modelo_data['recomendaciones']:
            print(f"\n🎯 Top 3 recomendaciones:")
            for i, rec in enumerate(modelo_data['recomendaciones'][:3], 1):
                print(f"  {i}. {rec['mensaje']} (Prioridad: {rec['prioridad']})")
        
        # Mostrar importancia de características si está disponible
        if 'feature_importance' in modelo_data:
            print(f"\n📊 Top características más importantes:")
            for idx, row in modelo_data['feature_importance'].head(3).iterrows():
                print(f"  {row['feature']}: {row['importance']:.3f}")
        
        return modelo_data
        
    except Exception as e:
        print(f"❌ Error cargando el modelo: {str(e)}")
        return None

print("✅ Función de carga de modelo definida")

## 🚀 Ejemplo de Uso del Modelo de Predicción

### Escenario 1: Entrenar nuevo modelo con datos de clustering

Para entrenar un nuevo modelo necesitas:
1. **df_resultados**: DataFrame con resultados del clustering (debe tener columnas: 'cluster', 'fecha_creacion')
2. **analisis_zonas**: DataFrame con análisis de zonas críticas (resultado del modelo de clustering)

### Escenario 2: Cargar modelo previamente entrenado

Si ya tienes un modelo entrenado, puedes cargarlo y usar sus predicciones.

In [ ]:
# 🔥 EJEMPLO 1: Entrenar nuevo modelo de predicción
# 
# Primero necesitas haber ejecutado el modelo de clustering para tener:
# - df_resultados: resultado del clustering
# - analisis_zonas: análisis de zonas críticas

# Simulación de datos para demostración (reemplaza con tus datos reales)
# df_resultados = pd.read_csv('tu_archivo_resultados_clustering.csv')
# analisis_zonas = pd.read_csv('tu_archivo_analisis_zonas.csv', index_col=0)

# NOTA: Descomenta las siguientes líneas cuando tengas los datos reales

# print("🚀 Iniciando entrenamiento del modelo de predicción...")
# resultado_prediccion = crear_modelo_prediccion_completo(
#     df_resultados=df_resultados,
#     analisis_zonas=analisis_zonas,
#     verbose=True
# )

# if resultado_prediccion['success']:
#     print(f"\n🎉 ¡Modelo de predicción entrenado exitosamente!")
#     print(f"📊 Modelo guardado en: {resultado_prediccion['modelo_path']}")
    
#     # Mostrar métricas
#     print(f"\n📈 Métricas del modelo:")
#     for metrica, valor in resultado_prediccion['metricas'].items():
#         print(f"  {metrica}: {valor:.4f}")
    
#     # Mostrar top recomendaciones
#     print(f"\n🎯 Top 3 recomendaciones:")
#     for i, rec in enumerate(resultado_prediccion['recomendaciones'][:3], 1):
#         print(f"  {i}. {rec['mensaje']} (Prioridad: {rec['prioridad']})")
    
#     # Generar visualizaciones
#     grafico_path = visualizar_predicciones(
#         resultado_prediccion['predicciones'],
#         analisis_zonas,
#         resultado_prediccion['recomendaciones']
#     )
    
#     print(f"📊 Gráficos generados en: {grafico_path}")
    
# else:
#     print(f"❌ Error en el entrenamiento: {resultado_prediccion['error']}")

print("💡 Descomenta el código anterior cuando tengas los datos del modelo de clustering")

In [ ]:
# 🔄 EJEMPLO 2: Cargar modelo de predicción existente
#
# Si ya tienes un modelo entrenado, puedes cargarlo y analizar sus resultados

# Ejemplo de carga (cambia la ruta por tu archivo real)
# modelo_path = "modelos/prediccion_reportes_20241212_143022.joblib"

# NOTA: Descomenta las siguientes líneas cuando tengas un modelo entrenado

# if os.path.exists(modelo_path):
#     print("📂 Cargando modelo de predicción existente...")
#     modelo_cargado = cargar_modelo_prediccion(modelo_path)
    
#     if modelo_cargado:
#         # Acceder a las predicciones
#         predicciones = modelo_cargado['predicciones_futuras']
#         recomendaciones = modelo_cargado['recomendaciones']
        
#         print(f"\n🔮 Predicciones disponibles para {len(predicciones)} zonas")
        
#         # Mostrar predicciones de una zona específica
#         if 'zona_0' in predicciones:
#             print(f"\n📊 Predicciones para Zona 0:")
#             for i, pred in enumerate(predicciones['zona_0'], 1):
#                 print(f"  Semana {i}: {pred['reportes_predichos']} reportes")
        
#         # Puedes generar visualizaciones si tienes analisis_zonas
#         # grafico_path = visualizar_predicciones(predicciones, analisis_zonas, recomendaciones)
        
# else:
#     print(f"❌ Archivo de modelo no encontrado: {modelo_path}")

print("💡 Descomenta el código anterior cuando tengas un modelo de predicción entrenado")

In [ ]:
def hacer_prediccion_personalizada(modelo_data, zona_info, semanas_futuras=4):
    """
    Hace predicciones personalizadas para una zona específica
    
    Parámetros:
    - modelo_data: Datos del modelo cargado
    - zona_info: Dict con información de la zona {'lat_centro', 'lng_centro', 'densidad', 'categoria_dominante'}
    - semanas_futuras: Número de semanas a predecir
    
    Retorna:
    - Lista de predicciones
    """
    try:
        modelo = modelo_data['modelo_prediccion']
        features_cols = modelo_data['feature_names']
        
        predicciones_personalizadas = []
        
        # Obtener fecha actual para calcular semanas futuras
        fecha_actual = datetime.now()
        semana_actual = fecha_actual.isocalendar().week
        mes_actual = fecha_actual.month
        año_actual = fecha_actual.year
        
        for i in range(1, semanas_futuras + 1):
            # Calcular fecha futura
            semana_pred = (semana_actual + i) % 52 or 52
            mes_pred = mes_actual if semana_pred >= semana_actual else (mes_actual % 12) + 1
            año_pred = año_actual if mes_pred >= mes_actual else año_actual + 1
            
            # Preparar características
            features_pred = {
                'semana_sin': np.sin(2 * np.pi * semana_pred / 52),
                'semana_cos': np.cos(2 * np.pi * semana_pred / 52),
                'mes_sin': np.sin(2 * np.pi * mes_pred / 12),
                'mes_cos': np.cos(2 * np.pi * mes_pred / 12),
                'lat_centro': zona_info['lat_centro'],
                'lng_centro': zona_info['lng_centro'],
                'densidad_zona': zona_info['densidad'],
                'categoria_dominante': zona_info.get('categoria_dominante', 0)
            }
            
            # Realizar predicción
            X_pred = pd.DataFrame([features_pred])
            reportes_pred = modelo.predict(X_pred)[0]
            
            predicciones_personalizadas.append({
                'semana': i,
                'año': año_pred,
                'mes': mes_pred,
                'semana_del_año': semana_pred,
                'reportes_predichos': max(0, round(reportes_pred)),
                'fecha_estimada': f"Semana {semana_pred}, {año_pred}"
            })
        
        return predicciones_personalizadas
        
    except Exception as e:
        print(f"❌ Error en predicción personalizada: {str(e)}")
        return []

# 🎯 EJEMPLO 3: Hacer predicción personalizada para una zona nueva
#
# Ejemplo de uso de predicción personalizada

# Definir información de una zona (ejemplo)
nueva_zona = {
    'lat_centro': -12.0464,  # Ejemplo: Lima, Perú
    'lng_centro': -77.0428,
    'densidad': 15,  # Reportes históricos promedio
    'categoria_dominante': 1  # ID de categoría dominante
}

# NOTA: Descomenta cuando tengas un modelo cargado
# if 'modelo_cargado' in locals() and modelo_cargado:
#     print("🎯 Haciendo predicción personalizada...")
#     predicciones_nuevas = hacer_prediccion_personalizada(modelo_cargado, nueva_zona)
    
#     if predicciones_nuevas:
#         print(f"\n📊 Predicciones para zona personalizada:")
#         total_predicho = 0
#         for pred in predicciones_nuevas:
#             print(f"  Semana {pred['semana']}: {pred['reportes_predichos']} reportes ({pred['fecha_estimada']})")
#             total_predicho += pred['reportes_predichos']
        
#         print(f"\n📈 Total predicho en {len(predicciones_nuevas)} semanas: {total_predicho} reportes")
        
#         # Generar recomendación simple
#         promedio_semanal = total_predicho / len(predicciones_nuevas)
#         if promedio_semanal > nueva_zona['densidad']:
#             print("⚠️  RECOMENDACIÓN: Se espera un incremento en reportes. Reforzar atención.")
#         elif promedio_semanal < nueva_zona['densidad'] * 0.5:
#             print("✅ RECOMENDACIÓN: Baja incidencia esperada. Monitoreo estándar.")
#         else:
#             print("📊 RECOMENDACIÓN: Actividad esperada dentro de rangos normales.")

print("✅ Función de predicción personalizada definida")
print("💡 Usa esta función para predecir reportes en zonas nuevas o modificadas")

## 📋 Resumen del Modelo de Predicción

### 🎯 **Funcionalidades Principales**

1. **`crear_modelo_prediccion_completo()`**: Función principal para entrenar un nuevo modelo
2. **`cargar_modelo_prediccion()`**: Carga un modelo previamente entrenado
3. **`hacer_prediccion_personalizada()`**: Hace predicciones para zonas nuevas
4. **`visualizar_predicciones()`**: Genera gráficos de análisis
5. **`generar_recomendaciones_inteligentes()`**: Sistema de recomendaciones automático

### 📊 **Características del Modelo**

- **Algoritmo**: Random Forest Regressor
- **Características temporales**: Patrones cíclicos de semana y mes
- **Características espaciales**: Coordenadas y densidad de zona
- **Predicciones**: 4 semanas futuras
- **Métricas**: MAE (Error Absoluto Medio) y R² (Coeficiente de determinación)

### ⚡ **Requisitos de Datos**

#### Para entrenar el modelo necesitas:
- **df_resultados**: DataFrame con columnas `['cluster', 'fecha_creacion', ...]`
- **analisis_zonas**: DataFrame con columnas `['lat_centro', 'lng_centro', 'densidad', ...]`

#### Los datos deben tener:
- Al menos 10 registros históricos con fechas válidas
- Datos de múltiples semanas/meses para capturar patrones temporales
- Información de zonas críticas previamente identificadas

### 🚀 **Flujo de Trabajo Recomendado**

1. **Ejecutar modelo de clustering** (desde `ejemplo.ipynb`)
2. **Obtener df_resultados y analisis_zonas**
3. **Ejecutar este modelo de predicción**
4. **Analizar recomendaciones y predicciones**
5. **Usar predicciones para planificación de recursos**

### 🔧 **Parámetros Configurables**

- **semanas_futuras**: Número de semanas a predecir (default: 4)
- **n_estimators**: Número de árboles en Random Forest (default: 50)
- **max_depth**: Profundidad máxima de árboles (default: 5)
- **verbose**: Mostrar mensajes de progreso (default: True)

### 💡 **Consejos de Uso**

- Más datos históricos = mejores predicciones
- Actualiza el modelo regularmente con nuevos datos
- Usa las recomendaciones para asignación de recursos
- Combina con el modelo de clustering para análisis completo